In [ ]:
# 1. 누락 데이터 처리
# 머신러닝 등 데이터 분석의 정확도는 분석 데이터의 품질에 의해 좌우
# 데이터 품질을 높이기 위해서는 누락 데이터, 중복 데이터 등 오류를 수정하고 분석 목적에 맞게 변형하는 과정이 필요
# 유효한 데이터 값이 존재하지 않는 누락 데이터를 NaN(Not a Number)
# 머신러닝 분석 모형에 데이터를 입력하기 전에는 [누락 데이터를 제거 / 다른 적절한 값으로 대체]하는 과정이 필요

In [2]:
# 누락 데이터 확인
# titanic 데이터의 deck에 NaN의 값이 있음

import seaborn as sns

df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
# 1. info() 메소드로 요약정보 출력하면 각 열에 속하는 데이터 중 유효한 값의 개수를 보여줌
# 총 891개의 데이터, deck 열에는 203개의 유효한 범주형 데이터가 있음.
# 따라서 688개의 누락데이터가 있음

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [4]:
# 2. value_counts()를 이용해서 누락데이터 개수 파악 가능

# DataFrame 객체.count() : 각 열의 데이터 개수를 시리즈 객체로 표시
# DataFrame 객체.value_counts() : 해당 열의 고유값의 개수를 세는데 사용 (dropna = False를 사용하면 Na의 개수도 세어줌)

nan_deck = df['deck'].value_counts(dropna = False)
print(nan_deck)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64


In [5]:
# 3. 누락데이터를 직접적으로 찾는 방법
# isnull() : 누락 데이터면 True를 반환하고, 유효한 데이터가 존재하면 False를 반환
# notnull() : 유효한 데이터가 존재하면 True를 반환, 누락 데이터면 False를 반환

# 누락데이터인 경우 True
print(df.head().isnull())
print('\n')

# 유효한 데이터인 경우 True
print(df.head().notnull())
print('\n')

# 각 열의 누락 데이터의 개수 구하기 (axis = 0) / 행으로 구할 경우에는 axis = 1
print(df.head().isnull().sum(axis=0))

   survived  pclass    sex    age  sibsp  parch   fare  embarked  class  \
0     False   False  False  False  False  False  False     False  False   
1     False   False  False  False  False  False  False     False  False   
2     False   False  False  False  False  False  False     False  False   
3     False   False  False  False  False  False  False     False  False   
4     False   False  False  False  False  False  False     False  False   

     who  adult_male   deck  embark_town  alive  alone  
0  False       False   True        False  False  False  
1  False       False  False        False  False  False  
2  False       False   True        False  False  False  
3  False       False  False        False  False  False  
4  False       False   True        False  False  False  


   survived  pclass   sex   age  sibsp  parch  fare  embarked  class   who  \
0      True    True  True  True   True   True  True      True   True  True   
1      True    True  True  True   True   True  Tr

In [16]:
# 누락 데이터 제거
# 누락 데이터가 들어 있는 열 또는 행을 삭제하는 방법
# 열 삭제 -> 특성(변수) 제거 / 행 삭제 -> 대상의 관측값(레코드) 삭제

import seaborn as sns

df = sns.load_dataset('titanic')

# for 반복문으로 각 열의 NaN 개수 계산하기
missing_df = df.isnull()
for col in missing_df.columns:
    missing_count = missing_df[col].value_counts()
    
    try:
        print(col, ': ', missing_count[True])
    except:
        print(col, ': ', 0)
        
print('\n')
        
# deck 데이터가 688명이나 누락되어 있음
# 따라서, deck 열의 누락 데이터를 삭제하여 분석에서 제외하는 것이 의미있음
# dropna() 메소드에 thresh = 500 옵션을 적용하여 NaN 값을 500개 이상 갖는 모든 열을 삭제

df_thresh = df.dropna(axis = 1, thresh = 500)
print(df_thresh.columns) # deck 열이 삭제된 것을 확인할 수 있음
print('\n')

# 승객의 나이 데이터는 177명이 누락되어 있음
# 만약, 승객의 나이가 데이터 분석의 중요한 변수라면 나이 데이터가 없는 승객의 레코드를 제거하는 것이 좋음
# dropna() 모소드에 subset을 age 열로 한정하면, age 열의 행 중 NaN값이 있는 모든 행을 삭제
# how = 'any' (하나라도 존재하면 삭제) / how ='all' (모든 데이터가 NaN일 경우 삭제)

df_age = df.dropna(subset = ['age'], how = 'any', axis = 0)
print(len(df_age)) # 714개의 행이 남음

survived :  0
pclass :  0
sex :  0
age :  177
sibsp :  0
parch :  0
fare :  0
embarked :  2
class :  0
who :  0
adult_male :  0
deck :  688
embark_town :  2
alive :  0
alone :  0


Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')


     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
885         0       3  female  39.0      0      5  29.1250        Q   Third   
886 

In [25]:
# 누락 데이터 치환 fillna()
# 데이터셋의 품질을 높일 목적으로 누락 데이터를 무작정 삭제한다면 어렵게 수집한 데이터를 활용하지 못함
# 데이터 분석의 정확도는 데이터 품질 외에도 데이터 양에도 상당한 영향을 받음
# 누락 데이터를 바꿔서 대체할 값으로는 데이터의 분포와 특성을 잘 나타낼 수 있는 평균값, 최빈값 등 활용
# 판다스 fillna() 메소드로 편하게 처리 가능, 새로운 객체 반환하기 때문에 inplace = True 옵션 필요


# age 열에 누락 데이터를 평균값으로 치환하기
import seaborn as sns

df = sns.load_dataset('titanic')

print(df['age'].head(10)) # 5행에 NaN 값 존재
print('\n')

# age 열의 NaN 값을 다른 나이 데이터의 평균으로 변경하기 (중간값을 사용하려면 median() 메소드를 적용)
mean_age = df['age'].mean(axis = 0) # mean() 메소드 사용 시 NaN을 제외한 평균값을 구해줌
df['age'].fillna(mean_age, inplace = True)

print(df['age'].head(10))
print('\n')


# 승선도시를 나타내는 embark_town 열에 있는 NaN을 승객들이 가장 많이 승선한 도시의 이름을 찾아 치환하기
import seaborn as sns

df = sns.load_dataset('titanic')

# embark_town 열의 829행의 NaN 데이터 출력
print(df['embark_town'][825:830])
print('\n')

# embark_town 열의 NaN 값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = df['embark_town'].value_counts(dropna = False).idxmax() # idxmax() : 최대값을 가지는 인덱스 라벨을 출력
print(most_freq)
print('\n')

df['embark_town'].fillna(most_freq, inplace = True)

print(df['embark_town'][825:830])

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: age, dtype: float64


0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: age, dtype: float64


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


Southampton


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object


In [ ]:
# 누락 데이터가 NaN으로 표시되지 않은 경우
# 누락 데이터가 NaN이 아닌 숫자 0이나 문자 -, ?와 같은 값으로 입력되기도 함
# 따라서, 누락 데이터를 다루기 위해서는 replace() 메소드를 활용하여 Numpy에서 지원하는 np.nan으로 변경해주는 것이 좋다

# 사용법 예시 
# import numpy as np
# df.replace('?', np.nan, inplace = True)

In [26]:
# 데이터셋의 특성상 서로 이웃하고 있는 데이터끼리 유사성을 가질 가능성이 높음
# 이럴 때에는 앞이나 뒤에서 이웃하고 있는 값으로 치환해주는 것이 좋음
# fillna() 메소드에 method = 'ffill' (직전 행 값으로 대체) / 'bfill' (다음 행 값으로 대체)를 사용하면 됨

import seaborn as sns

df = sns.load_dataset('titanic')

print(df['embark_town'][825:830])
print('\n')

df['embark_town'].fillna(method = 'ffill', inplace = True)
print(df['embark_town'][825:830])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
Name: embark_town, dtype: object


In [ ]:
# 2. 중복 데이터 처리
# 데이터프레임에서 각 행은 분석 대상이 갖고 있는 모든 속성에 대한 관측값을 뜻함
# 하나의 데이터셋에서 동일한 관측값이 2개 이상 중복되는 경우, 삭제해야 함
# 동일한 대상이 중복으로 존재하는 것이므로 분석 결과를 왜곡하기 때문

In [28]:
# 중복 데이터 확인
# duplicated() 메소드를 이용하여 행의 레코드가 중복되는지 여부를 확인
# 전에 나온 행들과 비교하여 중복되는 행이면 True, 처음 나오는 행에 대해서는 False를 반환


# 1. 데이터프레임 중복 데이터 확인
import pandas as pd

df = pd.DataFrame({'c1':['a', 'a', 'b', 'a', 'b'],
                   'c2':[1, 1, 1, 2, 2],
                   'c3':[1, 1, 2, 2, 2]})
print(df)
print('\n')

# 데이터프레임 전체 행 데이터 중에서 중복값 찾기
df_dup = df.duplicated()
print(df_dup)
print('\n')


# 2. 시리즈 객체 중복 데이터 확인
col_dup = df['c2'].duplicated()
print(col_dup)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


0    False
1     True
2    False
3    False
4    False
dtype: bool


0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool


In [31]:
# 중복 데이터 제거
# 중복 데이터를 제거하기 위해서는 drop_duplicates() 메소드가 있음
# 중복된 행을 제거하고 고유한 관측값을 가진 행들만 남김
# 원본 객체를 변경하기 위해서는 inplace = True라는 옵션을 추가


# 1. 데이터프레임 중복 데이터 제거
import pandas as pd

df = pd.DataFrame({'c1':['a', 'a', 'b', 'a', 'b'],
                   'c2':[1, 1, 1, 2, 2],
                   'c3':[1, 1, 2, 2, 2]})

print(df)
print('\n')

# 데이터 프레임에서 중복 행 제거
df2 = df.drop_duplicates()
print(df2)
print('\n')


# 2. 데이터프레임의 특정 열에 대한 중복 데이터 제거
# drop_duplicates() 메소드의 subset 옵션에 열 이름 리스트를 전달할 수 있음

df3 = df.drop_duplicates(subset = ['c2', 'c3'])
print(df3)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2


In [ ]:
# 3. 데이터 표준화
# 여러 곳에서 수집한 자료들은 단위 선택, 대소문자 구분, 약칭 활용 등 여러 가지 원인에 의해 다양한 형태로 표현
# 이처럼 동일한 대상을 표현하는 방법에 차이가 있으면 분석의 정확도는 현저히 낮아짐. 따라서 데이터 포맷을 일관성 있게 표준화하는 작업 필요

In [1]:
# 3-1. 단위 환산
# 특히, 외국 데이터를 가져오면 국내에서 잘 사용하지 않는 도량형 단위를 사용하는 경우가 많음
# 영미권에서 주로 사용하는 마일, 야드, 온스 등이 있는데 한국에서 사용하는 미터, 평, 그램 등으로 변환하는 것이 좋음

import pandas as pd

df = pd.read_csv('./auto-mpg.csv', header = None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model year', 'origin', 'name']
print(df.head(3))
print('\n')


# mpg(mile per gallon)을 kpl(kilometer per liter)로 변환 (mpg_to_kpl = 0.425)
mpg_to_kpl = 1.60934 / 3.78541

# mpg 열에 0.425를 곱한 결과를 새로운 열(kpl)에 추가
df['kpl'] = df['mpg'] * mpg_to_kpl
print(df.head(3))
print('\n')

# kpl 열을 소수점 아래 둘째자리에서 반올림
df['kpl'] = df['kpl'].round(2)
print(df.head(3))

    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  


    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   

   origin                       name       kpl  
0       1  chevrolet chevelle malibu  7.652571  
1       1          buick skylark 320  6.377143  
2       1         plymouth satellite  7.65257

In [12]:
# 3-2. 자료형 변환
# 숫자가 문자열(object)로 저장된 경우에 숫자형(int 혹은 float)으로 변환해야 함
# 먼저, dtypes 속성을 사용하여 데이터프레임을 구성하는 각 열의 자료형을 확인
# dtypes 대신 info() 메소드를 사용해도 확인할 수 있음

import pandas as pd

df = pd.read_csv('./auto-mpg.csv', header = None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model year', 'origin', 'name']

print(df.dtypes) # horsepower은 숫자형, model year, origin은 카테고리를 나타내는 범주형이 적절
print('\n')
print(df.info())
print('\n')

# 1. horsepower을 숫자형으로 변환
# 엔진 출력이 문자열로 저장된 이유 
# 열의 고유값을 출력해보자
print(df['horsepower'].unique()) # 고유값 중 문자 '?'로 섞여 데이터프레임으로 변환하는 과정에서 문자열로 인식됨
print('\n') 

# horsepower 열의 문자열 ?를 NaN으로 변환
# dropna 메소드로 NaN값이 들어있는 모든 행 삭제
# astype('float') 명령을 이용하여 문자열을 실수형으로 변환

import numpy as np
df['horsepower'].replace('?', np.nan, inplace = True)
df.dropna(subset = ['horsepower'], axis = 0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float') # astype()은 데이터프레임의 타입을 바꿀 때 사용

print(df['horsepower'].dtypes)
print('\n')



# 2. origin을 범주형으로 변환
# origin은 1, 2, 3 대신 실제 국가 이름으로 바꾸어주면 문자열을 나타내는 object로 자동 변경
print(df['origin'].unique())
print('\n')

df['origin'].replace({1:'USA', 2:'EU', 3:'JPN'}, inplace = True)
print(df['origin'].unique())
print('\n')
print(df['origin'].dtypes) # 문자열로 바뀜
print('\n')

# origin은 3개의 국가이름이 반복되므로, 이를 범주형 데이터로 표현하는 것이 효율적
# astype('category') 메소드를 이용하여 범주형 데이터로 변환
# 범주형을 다시 문자열로 변환하려면 astype('str') 메소드를 사용
df['origin'] = df['origin'].astype('category')
print(df['origin'].dtypes)
print('\n')


# 3. model_year을 범주형으로 변환
# 연도는 시간적인 순서는 의미 있으나 숫자의 상대적인 크기는 별 의미가 없음
# 따라서, 데이터는 숫자 형태를 갖더라도 자료형은 범주형으로 표현하는 것이 적절
print(df['model year'].sample(3)) # 무작위로 3개의 행을 출력하여 선택
print('\n')

df['model year'] = df['model year'].astype('category')
print(df['model year'].sample(3))

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model year        int64
origin            int64
name             object
dtype: object


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB
None


['130.0' '165.0' '150.0' '140.0' '198.0' '220.0' '215.0' '225.0' '190.0'
 '170.0' '160.0' '95.00' '97.00' '85.00' '88.00'

In [14]:
# 4. 범주형(카테고리) 데이터 처리

# 4-1. 구간 분할

# 넘파이 histogram() 이용하여 각 구간에 속하는 값의 개수 / 경계값 구하기
# 판다스 cut() 함수 이용

# 데이터 분석 알고리즘에 따라서는 연속 데이터를 그대로 사용하기 보다는 일정한 구간(bin)으로 나눠서 분석하는 것이 효율적인 경우가 있음
# 구간분할(binning) : 연속 변수를 일정한 구간으로 나누고, 각 구간을 범주형 이산 변수로 변환하는 과정
# 판다스 cut() 함수를 이용하면 연속 데이터를 여러 구간으로 나누고, 범주형 데이터로 변환 가능

# horsepower은 숫자로 표현하는 대신 저출력 / 보통출력 / 고출력 등 구간으로 나누어 표시하는 것이 효율적일 수 있음
# 3개의 구간으로 나누기 위해서는 총 4개의 경계값이 필요
# 경계값을 구하는 방법 : NumPy 라이브러리의 histogram() 함수를 활용, 나누려는 구간 개수를 bins 옵션에 입력하면 각 구간에 속하는 값의 개수(count)와 경계값 리스트(bin_dividers)를 반환

import pandas as pd
import numpy as np

df = pd.read_csv('./auto-mpg.csv', header = None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model year', 'origin', 'name']

# horsepower 열의 누락 데이터('?')를 삭제하고 실수형으로 변환
df['horsepower'].replace('?', np.nan, inplace = True)
df.dropna(subset = ['horsepower'], axis = 0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float')

# np.histogram 함수로 3개의 bin으로 구분할 경계값의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins = 3)
print(bin_dividers)
print('\n')

# 3개의 bin에 이름 지정
bin_names = ['저출력', '보통출력', '고출력']

# pd.cut 함수로 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x = df['horsepower'], # 데이터 배열
                      bins = bin_dividers, # 경계값 리스트
                      labels = bin_names, # bin 이름
                      include_lowest = True) # 각 구간의 낮은 경계값 포함 여부

print(df[['horsepower', 'hp_bin']].head(15))

[ 46.         107.33333333 168.66666667 230.        ]


    horsepower hp_bin
0        130.0   보통출력
1        165.0   보통출력
2        150.0   보통출력
3        150.0   보통출력
4        140.0   보통출력
5        198.0    고출력
6        220.0    고출력
7        215.0    고출력
8        225.0    고출력
9        190.0    고출력
10       170.0    고출력
11       160.0   보통출력
12       150.0   보통출력
13       225.0    고출력
14        95.0    저출력


In [15]:
# 4-2. 더미 변수

# 판다스 get_dummies() 함수 이용

# 앞에서 horsepower 열의 숫자형 연속 데이터를 hp_bin 열의 범주형 데이터로 변환
# 하지만, 카테고리를 나타내는 범주형 데이터를 머신러닝 알고리즘에 바로 사용할 수 없는 경우가 있어, 컴퓨터가 인식 가능한 입력값으로 변환해야 함
# 이런 경우, 숫자 0 또는 1로 표현되는 더미 변수(dummy variable)을 사용
# 여기서, 0과 1은 크고 작음을 나타내지 않고, 어떤 특성이 있는지 없는지 여부만을 표시
# 이처럼 범주형 데이터를 컴퓨터가 인식할 수 있도록 숫자 0,1로만 구성한다고 해서 원핫인코딩이라고도 부름

# 판다스 get_dummies() 함수를 사용하면, 범주형 변수의 모든 고유값을 각각 새로운 더미 변수로 변환
# hp_bin 열의 고유값 3개가 각각 새로운 더미변수 열의 이름이 됨
# 각 더미 변수가 본래 속해 있던 행에는 1이 입력되고, 속하지 않았던 다른 행에는 0이 입력됨

count, bin_dividers = np.histogram(df['horsepower'], bins = 3)

bin_names = ['저출력', '보통출력', '고출력']

df['hp_bin'] = pd.cut(x = df['horsepower'],
                      bins = bin_dividers,
                      labels = bin_names,
                      include_lowest = True)

# hp_bin 열의 범주형 데이터를 더미 변수로 변환
horsepower_dummies = pd.get_dummies(df['hp_bin'])
print(horsepower_dummies.head(15))

    저출력  보통출력  고출력
0     0     1    0
1     0     1    0
2     0     1    0
3     0     1    0
4     0     1    0
5     0     0    1
6     0     0    1
7     0     0    1
8     0     0    1
9     0     0    1
10    0     0    1
11    0     1    0
12    0     1    0
13    0     0    1
14    1     0    0


In [20]:
# sklearn 라이브러리를 이용해서 원핫인코딩을 편하게 처리할 수 있음
# hp_bin 열에 들어 있는 범주형 데이터를 0, 1을 원소로 갖는 원핫벡터로 변환
# 결과는 선형대수학에서 정의하는 희소행렬(sparse matrix)로 정리됨
# 예제에서는 1차원 벡터를 2차원 행렬로 변환한 후 다시 희소행렬로 변환함
# 희소행렬은 (행, 열) 좌표와 값의 형태로 정리
# 예를 들어 (0,1) 1.0은 0행 1열의 위치의 값은 1.0을 의미함

from sklearn import preprocessing

# 전처리를 위한 encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

# label_encoder로 문자열 범주를 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
print(onehot_labeled)
print(type(onehot_labeled))
print('\n')

# 2차원 행렬로 변경
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
print(onehot_reshaped)
print(type(onehot_reshaped))
print('\n')

# 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2]
<class 'numpy.ndarray'>


[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>


  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse.csr.csr_matrix'>


In [25]:
# 5. 정규화
# 숫자 데이터의 상대적인 크기 차이를 제거할 필요가 있음
# 각 열에 속하는 데이터 값을 동일한 크기 기준으로 나눈 비율로 나타내는 것을 정규화라고 함
# 정규화 과정을 거친 데이터의 범위는 0~1 또는 -1 ~ 1이 됨



# 1. 각 열의 데이터를 해당 열의 최대값(의 절대값)으로 나누는 방법
import pandas as pd
import numpy as np

df = pd.read_csv('./auto-mpg.csv', header = None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model year', 'origin', 'name']

# horsepower 열의 누락 데이터('?')를 삭제하고 실수형으로 변환
df['horsepower'].replace('?', np.nan, inplace = True)
df.dropna(subset = ['horsepower'], axis = 0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float')

# horsepower 열의 통계 요약 정보로 최대값(max) 확인
print(df.horsepower.describe())
print('\n')

# horsepower 열의 최대값의 절대값으로 모든 데이터를 나눠서 저장
df.horsepower = df.horsepower / abs(df.horsepower.max())
print(df.horsepower.head())
print('\n')
print(df.horsepower.describe())
print('\n')



# 2. 각 열의 데이터 중에서 최대값과 최소값을 뺀 값으로 나누는 방법
# 각 열의 데이터에서 해당 열의 최소값을 뺀 값을 분자로 하고, 해당 열의 최대값과 최소값의 차를 분모로 하는 수를 계산하면 가장 큰 값 역시 1이 됨
import pandas as pd
import numpy as np

df = pd.read_csv('./auto-mpg.csv', header = None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model year', 'origin', 'name']

# horsepower 열의 누락 데이터('?')를 삭제하고 실수형으로 변환
df['horsepower'].replace('?', np.nan, inplace = True)
df.dropna(subset = ['horsepower'], axis = 0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float')

# horsepower 열의 통계 요약 정보로 최대값, 최소값 확인
print(df.horsepower.describe())
print('\n')

# horsepower 열의 최대값의 절대값으로 모든 데이터를 나눠서 저장
min_x = df.horsepower - df.horsepower.min()
min_max = df.horsepower.max() - df.horsepower.min()
df.horsepower = min_x / min_max

print(df.horsepower.head())
print('\n')
print(df.horsepower.describe())

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64


0    0.565217
1    0.717391
2    0.652174
3    0.652174
4    0.608696
Name: horsepower, dtype: float64


count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64


count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64


0    0.456522
1    0.646739
2    0.565217
3    0.565217
4    0.510870
Name: horsepower, dtype: float64


count    392.000000
mean       0.317768
std        0.209191
min        0.000000
25%        0.157609
50%        0.258152
75%        0.434783
max        1.000000
Name: horsepower, dtype: float64


In [ ]:
# 6. 시계열 데이터
# 주식, 환율 등 금융 데이터를 다루기 위해 개발된 판다스는 시계열 데이터를 다루는 여러 가지 유용한 기능 제공
# 특히, 시계열 데이터를 데이터프레임의 행 인덱스로 사용하면, 시간으로 기록된 데이터를 분석하는 것이 매우 편리
# 판다스의 시간 표시 방식 중 자주 이용되는 2가지 유형 : 특정 시점 기록하는 Timestamp / 두 시점 사이의 일정한 기간을 나타내는 Period

In [30]:
# 6-1. 다른 자료형을 시계열 객체로 변환
# 우리가 접하는 많은 시간 데이터들은 별도의 시간 자료형(파이썬 datetime 라이브러리 등)으로 기록되지 않고, 문자열 or 숫자로 저장되는 경우가 많음
# 판다스는 다른 자료형으로 저장된 시간 데이터를 판다스 시계열 객체인 Timestamp로 변환하는 함수 제공


# 1. 문자열을 Timestamp로 변환
# 판다스 to_datetime() 함수를 사용하면 문자열 등 다른 자료형을 판다스 Timestamp를 나타내는 datetime64 자료형으로 변환 가능
import pandas as pd

df = pd.read_csv('./stock-data.csv')

print(df.head())
print('\n')
print(df.info()) # Date 열의 자료형이 문자열(object)임을 알 수 있음
print('\n')

df['new_Date'] = pd.to_datetime(df['Date']) # 문자열 데이터를 판다스 Timestamp로 변환하여 새로운 열로 추가

print(df.head())
print('\n')
print(df.info())
print('\n')
print(type(df['new_Date'][0])) # 해당 열의 개별 원소 데이터를 type() 함수로 확인하면 Timestamp 객체임을 알 수 있음
print('\n')

# 시계열 인덱스 지정
# 시계열 값을 행 인덱스로 지정하면 판다스는 DatetimeIndex로 저장함 (원래는 RangeIndex임)
# 시계열 인덱스 클래스를 지원하기 때문에 시간 순서에 맞춰 인덱싱, 슬라이싱 하기가 편함
df.set_index('new_Date', inplace = True)
df.drop('Date', axis = 1, inplace = True)

print(df.head())
print('\n')
print(df.info())
print('\n')



# 2. Timestamp를 Period로 변환
# 판다스 to_period() 함수를 이용하면 일정한 기간을 나타내는 Period 객체로 Timestamp 객체를 변환할 수 있음
# freq 옵션에 기준이 되는 기간을 설정
# freq 옵션을 'D'로 지정할 경우 1일의 기간, 'M'으로 지정할 경우 1개월의 기간, 'A'로 지정할 경우 1년의 기간을 나타냄
import pandas as pd

dates = ['2019-01-01', '2020-03-01', '2021-06-01']

# 문자열의 배열을 판다스 Timestamp로 변환
ts_dates = pd.to_datetime(dates)
print(ts_dates)
print('\n')

# Timestamp를 Period로 변환
pr_day = ts_dates.to_period(freq = 'D')
print(pr_day)
pr_month = ts_dates.to_period(freq = 'M')
print(pr_month)
pr_year = ts_dates.to_period(freq = 'A')
print(pr_year)

         Date  Close  Start   High    Low  Volume
0  2018-07-02  10100  10850  10900  10000  137977
1  2018-06-29  10700  10550  10900   9990  170253
2  2018-06-28  10400  10900  10950  10150  155769
3  2018-06-27  10900  10800  11050  10500  133548
4  2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB
None


         Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018

In [36]:
# 6-2. 시계열 데이터 만들기

# 1. Timestamp 배열
# 판다스 date_range() 함수를 사용하면 여러 개의 날짜가 들어 있는 배열 형태의 시계열 데이터를 만들 수 있음
# 파이썬 range() 함수로 숫자 배열을 만드는 것과 비슷
import pandas as pd

ts_ms = pd.date_range(start = '2019-01-01', # 날짜 범위 시작
                      end = None, # 날짜 범위 끝
                      periods = 6, # 생성할 timestamp 개수
                      freq = 'MS', # 시간 간격 (MS : 월의 시작일)
                      tz = 'Asia/Seoul') # 시간대(timezone)

print(ts_ms)
print('\n')

# 시간 간격을 다르게 설정
# freq = 'M'으로 입력하면 월의 마지막 날짜를 생성
# freq = '3M'으로 하며 3개월 간격의 마지막 날짜를 나타냄
ts_me = pd.date_range('2019-01-01', periods = 6,
                      freq = 'M',
                      tz = 'Asia/Seoul')
print(ts_me)
print('\n')

ts_3m = pd.date_range('2019-01-01', periods = 5,
                      freq = '3M',
                      tz = 'Asia/Seoul')
print(ts_3m)
print('\n')




# 2. Period 배열
# 판다스 period_range() 함수는 여러 개의 기간이 들어 있는 시계열 데이터를 만듦
# PeriodIndex의 원소 '2019-01'은 2019년 1월의 전체 기간을 의미
import pandas as pd
pr_m = pd.period_range(start = '2019-01-01', # 날짜 범위 시작
                       end = None, # 날짜 범위 끝
                       periods = 3, # 생성할 period 개수
                       freq = 'M') # 기간의 길이(M : 월)
print(pr_m)
print('\n')

# 기간을 다르게 설정
# freq = 'H' 옵션은 1시간 간격을 나타냄
# freq = '2H' 옵션은 2시간 간격을 나타냄
pr_h = pd.period_range(start = '2019-01-01',
                       end = None,
                       periods = 3,
                       freq = 'H')
print(pr_h)
print('\n')


pr_2h = pd.period_range(start = '2019-01-01',
                        end = None,
                        periods = 3,
                        freq = '2H')
print(pr_2h)

DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')


PeriodIndex(['2019-01', '2019-02', '2019-03'], dtype='period[M]')


PeriodIndex(['2019-01-01 00:00', '2019-01-01 01:00', '2019-01-01 02:00'], dtype='period[H]')


PeriodIndex(['2019

In [41]:
# 6-3. 시계열 데이터 활용

# 1. 날짜 데이터 분리

# 연 - 월 -일 날짜 데이터에서 일부를 분리하여 추출 가능
# dt 속성을 이용하여 개별적으로 추출 가능

import pandas as pd

df = pd.read_csv('./stock-data.csv')

df['new_Date'] = pd.to_datetime(df['Date']) # 문자열인 날짜 데이터를 판다스 Timestamp로 변환하여 새로운 열로 추가
print(df.head())
print('\n')

# dt 속성을 이용하여 new_Date 열의 연-월-일 정보를 년, 월, 일로 구분
df['Year'] = df['new_Date'].dt.year
df['Month'] = df['new_Date'].dt.month
df['Day'] = df['new_Date'].dt.day
print(df.head())
print('\n')

# Timestamp 객체를 Period 객체로 변환하는 to_period() 메소드를 적용하여, 연-월-일 중에서 연-월 또는 연도를 추출
df['Date_yr'] = df['new_Date'].dt.to_period(freq = 'A')
df['Date_m'] = df['new_Date'].dt.to_period(freq = 'M')
print(df.head())
print('\n')

# 추출한 날짜 정보를 데이터프레임의 행 인덱스로 지정 가능
df.set_index('Date_m', inplace = True)
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  Day
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7    2
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   26


         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  \
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7   
1 

In [65]:
# 2. 날짜 인덱스 활용
# Timestamp로 구성된 열을 행 인덱스로 지정하면 DatetimeIndex라는 고유 속성으로 변환됨
# 마찬가지로, Period로 구성된 열을 행 인덱스로 지정하면 PeriodIndex라는 속성을 가짐
# 이와 같은 날짜 인덱스를 활용하면 시계열 데이터에 대한 인덱싱과 슬라이싱이 편함

import pandas as pd

df = pd.read_csv('stock-data.csv')

df['new_Date'] = pd.to_datetime(df['Date'])
df.set_index('new_Date', inplace = True)

print(df.head())
print('\n')
print(df.index)
print('\n')

# 날짜 인덱스를 사용하는 장점은 연 - 월 - 일 중에서 내가 필요로 하는 레벨을 선택적으로 인덱싱 가능
# 연도, 연-월, 연-월-일을 기준으로 선택 가능
# 날짜 범위로 슬라이싱 추출도 가능
df_y = df.loc['2018'] # 2018년인 행만 추출
print(df_y.head())
print('\n')

df_ym = df.loc['2018-07']
print(df_ym)
print('\n')

df_ym_cols = df.loc['2018-07', 'Start':'High'] # 원소 선택
print(df_ym_cols)
print('\n')

df_ymd = df.loc['2018-07-02']
print(df_ymd)
print('\n')

df_ymd_range = df.loc['2018-06-25':'2018-06-20'] # 날짜 범위 지정, 빈 데이터프레임 출력, 오류
print(df_ymd_range)
print('\n')

# Timestamp 객체로 표시된 두 날짜 사이의 시간 간격 구하기
# 어떤 날짜로부터 경과한 날짜를 계산하여 행 인덱스로 지정하고, 데이터를 선택

# 시간 간격 계산, 최근 180일 ~ 189일 사이의 값들만 선택
today = pd.to_datetime('2018-12-25') # 기준일 생성
df['time_delta'] = today - df.index # 날짜 차이 계산
df.set_index('time_delta', inplace = True) # 행 인덱스로 지정
df_180 = df.loc['180 days':'189 days']
print(df_180)

                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


DatetimeIndex(['2018-07-02', '2018-06-29', '2018-06-28', '2018-06-27',
               '2018-06-26', '2018-06-25', '2018-06-22', '2018-06-21',
               '2018-06-20', '2018-06-19', '2018-06-18', '2018-06-15',
               '2018-06-14', '2018-06-12', '2018-06-11', '2018-06-08',
               '2018-06-07', '2018-06-05', '2018-06-04', '2018-06-01'],
              dtype='datetime64[ns]', name='new_Date', freq=None)


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900